# SharePoint

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import SharepointTool, MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [2]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup the SharePoint Tool

In [3]:
# Enter SharePoint connection name here
sharepoint_connection = "sharepoint"  # Replace with your SharePoint connection name

# Extract the connection list.
conn_list = project_client.connections.list()
sharepoint_connection_id = ""

# Find the connection IDs for SharePoint
for conn in conn_list:
    if conn.name == sharepoint_connection:
        sharepoint_connection_id = conn.id


# Print the connection names
print(f"SharePoint Connection ID: {sharepoint_connection_id}")

# Initialize the SharePoint tools
sharepoint_tool = SharepointTool(connection_id=sharepoint_connection_id)

SharePoint Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/AzureStandardAgent/providers/Microsoft.CognitiveServices/accounts/foundykx3l/projects/projectkx3l/connections/sharepoint


## Create the agent

In [4]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-sharepoint-agent",  # Name of the agent
    instructions="You are a helpful agent and can search information from SharePoint",  # Instructions for the agent
    tools=sharepoint_tool.definitions
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_v6N1IRmLZRV7sDZKZzJluvMC


## Create thread

In [5]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_G4d3UtBt3bP2nm7IRXFiW2Qb


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [7]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [8]:
user_input = "Provide details on the Northwind Standard Plan"
run_agent(user_input)

Created message, ID: msg_HrCJClEfjoxVjdiX5U7IQLis
Run finished with status: completed
Agent Response: The Northwind Standard Plan offers comprehensive health coverage with the following key features and costs:

### Costs:
1. **Premiums**: Monthly payment based on age, health, and selected coverage.
2. **Deductibles**: $2,000 per person annually before coverage starts.
3. **Co-payments and Co-insurance**:
   - $30 for primary care visits
   - $50 for specialist visits
   - 20% co-insurance after meeting the deductible
4. **Out-of-Pocket Maximum**: $6,000 per person annually, after which the plan covers 100% of expenses for the year.

### Covered Services:
1. **Hospitals**: Access to various in-network hospitals and comprehensive maternity care including prenatal care, delivery, and post-delivery care.
2. **Prescription Drugs**: Coverage for brand name and generic medications, as well as over-the-counter drugs when prescribed. Preferred pharmacies provide additional cost savings.
3. **Pr

In [9]:
user_input = "Provide a tabular comparison between the Northwind Standard and Northwind Healthplus plans"
run_agent(user_input)

Created message, ID: msg_yltaK2DntBpu0XQfTjzHTflK
Run finished with status: completed
Agent Response: Here is a tabular comparison between the Northwind Standard and Northwind Healthplus plans:

| Feature                          | Northwind Standard Plan                              | Northwind Healthplus Plan                           |
|----------------------------------|------------------------------------------------------|----------------------------------------------------|
| **Premiums**                     | Based on age, health, and selected coverage          | Based on plan type; employer pays part of the premium |
| **Deductibles**                  | $2,000 per person per year                           | Varies; higher for out-of-network providers        |
| **Co-payments**                  | $30 for primary care, $50 for specialists            | Varies by service and provider type                |
| **Co-insurance**                 | 20% after deductible                   

## Cleanup resources

In [10]:
# Cleanup resources
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted agent
